In [260]:
from symbol import import_as_name
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import pickle
import re
from nltk.stem import *
from gensim.parsing.preprocessing import remove_stopwords
from utils import *
import xmltodict
import pandas as pd
import string
%matplotlib inline
plt.style.use('ggplot')

In [261]:
# load index into memory
with open('collections/trec.english_stemmer.dat', 'rb') as f:
    data = pickle.load(f)
len(data)
len(data['incom'].keys() & data['come'].keys() & data['like'].keys() )

16

In [262]:
with open('collections/trec.sample.xml', 'r', encoding='utf-8') as f:
    xml_file = f.read()
# load total text in the dict_file
dict_file = xmltodict.parse(xml_file)
dict_file = dict_file['document']['DOC']
N = len(dict_file)
# pprint.pprint(dict_file[999]) # index not equal to DOCNO

In [263]:
def compute_tf(content, document, data):
    terms = content.split()
    tf_list = []
    for t in terms:
        tf = len(data[t][str(document)])
        tf_list.append(tf)
    return tf_list

def compute_df(content, data):
    terms = content.split()
    df_list = []
    for t in terms:
        df = len(data[t])
        df_list.append(df)
    return df_list


    

content = 'middl east peac'
res = compute_tf(content, '219', data)
res



[1, 1, 3]

In [264]:
def retrival_search(content, data, N):
    doc_dic, sentence = search(content, data)
    w_matrix = np.zeros((len(sentence.split()), len(doc_dic)))
    df = compute_df(sentence, data)
    # df = np.array(df)[:,None]
    for column, doc in enumerate(doc_dic):
        tf = compute_tf(sentence, doc, data)
        # tf = np.array(tf)[:,None]
        w_matrix[:,column] = (1 + np.log10(tf))*(np.log10(N)-np.log10(df))
    score_q = np.sum(w_matrix,axis=0)[:,None]
    doc_array = np.array([doc for doc in doc_dic])[:,None]
    score_q = np.concatenate((doc_array, score_q), axis=1)
    score_q = score_q[score_q[:,1].argsort()][::-1]
    return score_q
content = '"middle east" AND peace'

retrival_search(content, data, N)
        

array([['3549', '7.1810539886606755'],
       ['305', '6.975317638790024'],
       ['288', '6.7995569814859165'],
       ['223', '6.022628887325533'],
       ['219', '4.947500168980254'],
       ['3762', '4.947500168980254'],
       ['3663', '4.626325782384706'],
       ['3766', '4.07727454200674']], dtype='<U32')

In [265]:
def process_query(query_path):
    punct = re.compile("[^\w\s]") # regax of token
    with open(query_path, 'r', encoding='utf-8') as f:
        querys = f.readlines()
    query_list = []
    for line in querys:
        line = re.split(r"^\d{1,}", line)[1]
        line = re.split(r"$[\n]", line)[0]
        line = re.sub(punct, "", line) # tokenization
        line = line.strip()
        query_list.append(line)
    return query_list
        
    
query_path = 'querys.txt'
process_query(query_path)

['income tax reduction',
 'peace in the Middle East',
 'unemployment rate in UK',
 'industry in scotland',
 'the industries of computers',
 'Microsoft Windows',
 'stock market in Japan',
 'the education with computers',
 'health industry',
 'campaigns of political parties']

In [266]:
def query_search(query_path, data):
    query_list = process_query(query_path)
    rank_table = []
    for index, query in enumerate(query_list):
        score_q = retrival_search(query, data, N)
        for score_qd in score_q:
            rank_table.append([index+1, str(score_qd[0]), float(score_qd[1])])
    return rank_table
query_path = 'querys.txt'
query_search(query_path, data)

[[1, '65', 5.266347855213409],
 [1, '3533', 5.1829337881155215],
 [1, '3562', 3.8102158712166783],
 [1, '361', 3.5341067552200993],
 [1, '141', 3.5341067552200993],
 [1, '3608', 3.485384803595027],
 [1, '3599', 3.1705804307459706],
 [1, '3387', 3.1705804307459706],
 [1, '3829', 3.1705804307459706],
 [2, '3549', 7.1810539886606755],
 [2, '305', 6.975317638790024],
 [2, '288', 6.799556981485917],
 [2, '223', 6.022628887325533],
 [2, '219', 4.947500168980254],
 [2, '3762', 4.947500168980254],
 [2, '3663', 4.626325782384706],
 [2, '3766', 4.07727454200674],
 [3, '3354', 4.619687808799235],
 [3, '3345', 4.512580806697169],
 [3, '268', 3.5844170407006892],
 [3, '328', 3.5048378185057216],
 [3, '21', 3.4207444223145487],
 [3, '304', 3.3939997977728997],
 [3, '313', 3.364704376161913],
 [3, '55', 3.0639336602268776],
 [3, '3789', 2.726253988972256],
 [3, '3506', 2.656960447518389],
 [3, '3653', 2.513201837201487],
 [3, '46', 2.513201837201487],
 [4, '3938', 3.230598089758382],
 [4, '351', 3.23

In [267]:
def write_query_results(results, path):
    with open(path, 'w', encoding='utf-8') as f:
        for line in results:
            f.write(str(line)[1:-1]+'\n')
result_path = 'tfidf.results'
results = query_search(query_path,data)
write_query_results(results, result_path)